In [ ]:
from sklearn.model_selection import train_test_split
import random
import shutil
import os
import torch
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
import numpy as np

In [ ]:
%ls

In [ ]:
%cd /cluster/home/sondrmka/

In [ ]:
#%cd /cluster/projects/vc/courses/TDT17/2022/open/RDD2022 (could load data from here)

In [ ]:
# os.makedirs('./.kaggle')
os.listdir()

In [ ]:
%cd /cluster/work/sondrmka/TDT17

%ls

In [ ]:
# os.makedirs('./input')
# os.makedirs('./output')

In [ ]:
%pip install python-dotenv --user
%pip install kaggle --user

In [ ]:
# shutil.move("kaggle.json", "/cluster/home/sondrmka/.kaggle")
%ls

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

In [ ]:
#!chmod 600 /cluster/home/sondrmka/.kaggle/kaggle.json

In [ ]:
dataset = "rdd2022-norway-yolo"

# Download dataset
api.dataset_download_files(f"paalmorholmen/{dataset}", path="./input", quiet=False)

In [ ]:
import zipfile

with zipfile.ZipFile(f"./input/{dataset}.zip") as file:
    file.extractall("./data")

os.remove(f"./input/{dataset}.zip")

In [ ]:
%cd /cluster/work/sondrmka/TDT17/output

In [ ]:
# os.makedirs('./images/train')
# os.makedirs('./images/val')
# os.makedirs('./images/test')
# os.makedirs('./labels/train')
# os.makedirs('./labels/val')
# os.makedirs('./labels/test')

In [ ]:
%cd /cluster/work/sondrmka/TDT17/data

In [ ]:
# Read images and labels
images = [os.path.join("./images", x) for x in os.listdir("./images")]
labels = [
    os.path.join("./labels", x) for x in os.listdir("./labels") if x[-3:] == "txt"
]

images.sort()
labels.sort()

annotated_img_ids = [img[7:-4] for img in labels]
filtered_images = labels.copy()

filtered_images = [
    img.replace("txt", "jpg").replace("labels", "images") for img in filtered_images
]

# Split the dataset into train-valid-test splits
train_images, val_images, train_labels, val_labels = train_test_split(
    filtered_images, labels, test_size=0.2, random_state=1
)

# using all images for training
# val_images, test_images, val_labels, test_labels = train_test_split(
# val_images, val_labels, test_size=0.5, random_state=1
# )

print(len(train_images))
print(len(val_images))

In [ ]:
# Utility function to move images
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            # print(f)
            pass
            # assert False


# Move the splits into their folders
move_files_to_folder(train_images, "../output/images/train")
move_files_to_folder(val_images, "../output/images/val/")
# move_files_to_folder(test_images, "../output/images/test/")
move_files_to_folder(train_labels, "../output/labels/train/")
move_files_to_folder(val_labels, "../output/labels/val/")
# move_files_to_folder(test_labels, "../output/labels/test/")

In [ ]:
%cd /cluster/work/sondrmka/TDT17/

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git ./yolov7

In [ ]:
%cd /cluster/work/sondrmka/TDT17/yolov7/

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7x_training.pt

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6_training.pt

In [ ]:
%cd /cluster/work/sondrmka/TDT17/yolov7/data

In [ ]:
train_path = "/cluster/work/sondrmka/TDT17/output/images/train/"
val_path = "/cluster/work/sondrmka/TDT17/output/images/val/"

# number of classes
nc = 4

# class names
names = ["D00", "D10", "D40", "D20"]

In [ ]:
import yaml

with open("rdd.yaml", "w") as file:
    yaml.dump(
        {
            "train": train_path,
            "val": val_path,
            "nc": nc,
            "names": [f"{name}" for name in names],
        },
        stream=file,
        default_flow_style=None,
    )

In [ ]:
%cd /cluster/work/sondrmka/TDT17/yolov7/

In [ ]:
%cd /cluster/work/sondrmka/TDT17/output/images/train

In [ ]:
training_images = os.listdir()
orig_img = Image.open(training_images[228])
plt.imshow(orig_img)
plt.show()

In [ ]:
new_img = ...
plt.imshow(new_img)
plt.show()

In [ ]:
def plot(imgs, with_orig=True, row_title=None, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0]) + with_orig
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        row = [orig_img] + row if with_orig else row
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    if with_orig:
        axs[0, 0].set(title="Original image")
        axs[0, 0].title.set_size(8)
    if row_title is not None:
        for row_idx in range(num_rows):
            axs[row_idx, 0].set(ylabel=row_title[row_idx])

    plt.tight_layout()

In [ ]:
center_crops = [
    T.CenterCrop(size=size)(orig_img) for size in (30, 50, 100, orig_img.size)
]

plot(center_crops)
plt.show()

In [ ]:
def change_annotation_index(detection_file, output_path, label_path):
    classes = ["D00", "D10", "D40", "D20"]
    new_annotation_index = {"0": "1", "1": "2", "2": "4", "3": "3"}
    new_detections = []

    with open(label_path + detection_file, "r") as file:
        Lines = file.readlines()
        for line in Lines:
            new_string = new_annotation_index.get(line[0]) + " " + line[1:].strip()
            new_detections.append(new_string)

    detection_file = detection_file.replace(".txt", ".jpg")
    # print(new_detections)
    print(detection_file)
    print(output_path)
    with open(output_path + detection_file, "w") as file:
        for line in new_detections:
            file.write(line)

In [ ]:
def export_new_labels_to_output(run, output_path):
    label_path = (
        "/cluster/work/sondrmka/TDT17/yolov7/runs/detect/" + str(run) + "/labels/"
    )
    label_files = os.listdir(label_path)

    path = output_path + run + "/labels/"

    for file in label_files:
        change_annotation_index(file, path, label_path)

In [ ]:
export_new_labels_to_output("exp3", "/cluster/work/sondrmka/TDT17/output/runs/")